In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
import string
from dateutil import parser

In [2]:
def get_soup(start_url):
    response = requests.get(start_url)
    page = response.text
    return BeautifulSoup(page, "lxml")

### A function to create list of Billboard Hot 100 urls that catalogue each song going back to 1958

In [3]:
from datetime import date, datetime, timedelta

def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

urls = []
for suffix in perdelta(date(1958, 8, 9), date(2017, 5, 20), timedelta(days=7)):
    prefix = 'http://www.billboard.com/charts/hot-100/'
    link = prefix + str(suffix)
    urls.append(link)


In [4]:
urls[:3]

['http://www.billboard.com/charts/hot-100/1958-08-09',
 'http://www.billboard.com/charts/hot-100/1958-08-16',
 'http://www.billboard.com/charts/hot-100/1958-08-23']

In [230]:
def get_artist(soup):
    artist = soup.find_all(class_="chart-row__artist")
    artists = []
    for item in artist:
        item = item.text.strip()
        artists.append(item)
    return artists

In [231]:
def get_track(soup):
    track = soup.find_all(class_="chart-row__song")
    tracks = []
    for item in track:
        item = item.text.strip()
        tracks.append(item)
    return tracks


### Get artist and track title from each url. Append to list of dataframes, merge the dataframes and write to a flat file

In [232]:

frames = []
billboard_dict = {}
for url in urls:
    soup = get_soup(url)
    artist = get_artist(soup)
    track = get_track(soup)
    dates = [parser.parse(url.split('/')[5])]*len(artist)
    df = pd.DataFrame({'artist': artist, 'track': track, 'publish_date': dates})

    frames.append(df)
    

In [233]:
df_merge = pd.concat(frames).reset_index(drop=True)

In [237]:
# import pickle
# with open('dataframes/billboard_rawdf.pkl', 'wb') as picklefile:
#     pickle.dump(df_merge, picklefile)

In [ ]:
# with open('dataframes/billboard_rawdf.pkl', 'rb') as picklefile:
#     df_bb = pickle.load(picklefile, encoding="utf-8")

In [ ]:
df2 = df_bb.drop_duplicates(['track'], keep='first').reset_index(drop=True)

In [ ]:
# with open('dataframes/billboard_unique.pkl', 'wb') as picklefile:
#     pickle.dump(df2, picklefile)